In [18]:
import cv2
import math
import numpy as np

img = cv2.imread('lena.png')
height, width, channels = img.shape

# Reduce img channels from 3 to 1 (greyscale)
img = img[:,:,0]
# Create DCT and IDCT matrices, Quantised Result, De-quantised Result
DCT = np.zeros((height, width))
IDCT = np.zeros((height, width))
Q_result = np.zeros((height, width))
DeQ_result = np.zeros((height, width))

# Quantisation Table
Q_table = np.array([[16, 11, 10, 16,  24, 40,  51,   61],
                    [12, 12, 14, 19,  26, 58,  60,   55],
                    [14, 13, 16, 24,  40, 57,  69,   56],
                    [14, 17, 22, 29,  51, 87,  80,   62],
                    [18, 22, 37, 56,  68, 109, 103,  77],
                    [24, 35, 55, 64,  81, 104, 113,  92],
                    [49, 64, 78, 87, 103, 121, 120, 101],
                    [72, 92, 95, 98, 112, 100, 103,  99]])

'''Q_table1 = np.array([[,,,,,,,],
                        [,,,,,,,],
                        [,,,,,,,],
                        [,,,,,,,],
                        [,,,,,,,],
                        [,,,,,,,],
                        [,,,,,,,],
                        [,,,,,,,]])'''

# List to store C() function
temp = (2)**(1/2.0)/(8)**(1/2.0)
C = [1/(8)**(1/2.0), temp, temp, temp, temp, temp, temp, temp]

# Precalculation of Cosines combinations, format:[x/y, u/v], range:[0, 0] -> [7, 7]
cos = np.zeros((8, 8))
for i in range (0, 8):
    for j in range (0, 8):
        cos[i, j] = math.cos((2*i+1)*j*math.pi/16.0)

# Do DCT and save in DCT matrix
for row in range (0, height, 8):
    string = '{0} rows left'.format(height-row)
    print(string)
    for col in range (0, width, 8):
        for u in range (row, row+8):
            for v in range (col, col+8):
                for x in range (row, row+8):
                    for y in range (col, col+8):
                        DCT[u, v] += img[x, y] * cos[x%8, u%8] * cos[y%8, v%8]
                DCT[u, v] *= C[u%8]*C[v%8] 
print('DCT transformation done!')

# Quantisation
for row in range (0, height, 8):
    for col in range (0, width, 8):
        for i in range (row, row+8):
            for j in range (col, col+8):
                Q_result[i, j] = DCT[i, j]/Q_table[i%8, j%8]
print('Quantisation done!')

# De-quantisation
for row in range (0, height, 8):
    for col in range (0, width, 8):
        for i in range (row, row+8):
            for j in range (col, col+8):
                DeQ_result[i, j] = Q_result[i, j]*Q_table[i%8, j%8]
print('De-quantisation done!')

# Do Inverse DCT and save in IDCT matrix
for row in range (0, height, 8):
    string = '{0} rows left'.format(height-row)
    print(string)
    for col in range (0, width, 8):
        for x in range (row, row+8):
            for y in range (col, col+8):
                for u in range (row, row+8):
                    for v in range (col, col+8):
                        IDCT[x, y] += C[u%8]*C[v%8] * DeQ_result[u, v] * cos[x%8, u%8] * cos[y%8, v%8]
print('IDCT transformation done!')
                       
# Normalise DCT coefficient values to greyscale 0-255 for image output (originally 0-2040, dtype=float)
DCT[:,:] = DCT[:,:]/2040      
# Change IDCT datatype to 8-bit unsinged integer for image output (originally 0-255, dtype=float)
IDCT = IDCT.astype(np.uint8)

512 rows left
504 rows left
496 rows left
488 rows left
480 rows left
472 rows left
464 rows left
456 rows left
448 rows left
440 rows left
432 rows left
424 rows left
416 rows left
408 rows left
400 rows left
392 rows left
384 rows left
376 rows left
368 rows left
360 rows left
352 rows left
344 rows left
336 rows left
328 rows left
320 rows left
312 rows left
304 rows left
296 rows left
288 rows left
280 rows left
272 rows left
264 rows left
256 rows left
248 rows left
240 rows left
232 rows left
224 rows left
216 rows left
208 rows left
200 rows left
192 rows left
184 rows left
176 rows left
168 rows left
160 rows left
152 rows left
144 rows left
136 rows left
128 rows left
120 rows left
112 rows left
104 rows left
96 rows left
88 rows left
80 rows left
72 rows left
64 rows left
56 rows left
48 rows left
40 rows left
32 rows left
24 rows left
16 rows left
8 rows left
DCT transformation done!
Quantisation done!
De-quantisation done!
512 rows left
504 rows left
496 rows left
488 rows 

In [19]:
# Calculate Mean Squared Error for PSNR
MSE = ( img[:,:] - IDCT[:,:] )**(2)
MSE = np.sum(MSE) / (height*width)

# Calculate PSNR value 
PSNR = 10*math.log((255*255/MSE), 10)
print('The PSNR value is ', PSNR)

# Calculate error image 
error_img = abs( img[:,:] - IDCT[:,:] )
# The difference between compressed img and original img is very very small  
# So thresholding is done to hightlight the positions of changed values
ret, error_img = cv2.threshold(error_img, 0, 255, cv2.THRESH_BINARY)

print('IDCT greyscale image matrix\n', IDCT)
print('\nOriginal greyscale image matrix\n', img)
print('\nError image matrix\n', error_img)
cv2.imshow('Original', img)
cv2.imshow('DCT', DCT)
cv2.imshow('IDCT', IDCT)
cv2.imshow('Error Image', error_img)
cv2.imwrite('Compressed.png', IDCT)
cv2.imwrite('DCT.png', DCT)
cv2.imwrite('Error.png', error_img)

cv2.waitKey(0)
cv2.waitKey(1)
cv2.destroyAllWindows
for i in range (1,10):
    cv2.waitKey(i)


The PSNR value is  49.702313084604754
IDCT greyscale image matrix
 [[161 162 161 ... 169 155 128]
 [161 161 161 ... 169 154 127]
 [161 161 161 ... 169 154 127]
 ...
 [ 42  42  49 ... 103  99  97]
 [ 44  44  54 ... 104 105 107]
 [ 43  43  54 ... 103 104 107]]

Original greyscale image matrix
 [[162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 [162 162 162 ... 170 155 128]
 ...
 [ 43  43  50 ... 104 100  98]
 [ 44  44  55 ... 104 105 108]
 [ 44  44  55 ... 104 105 108]]

Error image matrix
 [[255   0 255 ... 255   0   0]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [  0   0 255 ...   0   0 255]
 [255 255 255 ... 255 255 255]]
